In [11]:
import numpy as np
import nibabel as nib  # For saving NIfTI files (if needed)
from scipy.ndimage import zoom  # For resizing

def generate_synthetic_mri_data(num_samples=358, img_size=(64, 64, 64), ad_proportion=0.5):
    """
    Generates synthetic 3D MRI-like data for Alzheimer's Disease (AD) classification.

    This function creates simplified, random data that mimics some general
    characteristics of MRI scans but does NOT represent real brain anatomy.

    Args:
        num_samples (int): Total number of samples to generate.
        img_size (tuple): Size of the 3D image (height, width, depth).
        ad_proportion (float): Proportion of samples that are AD (0 to 1).

    Returns:
        tuple: (images, labels) where images is a numpy array of shape
               (num_samples, img_size[0], img_size[1], img_size[2]) and
               labels is a numpy array of shape (num_samples,).
    """

    num_ad = int(num_samples * ad_proportion)
    num_control = num_samples - num_ad

    images = np.random.rand(num_samples, img_size[0], img_size[1], img_size[2])  # Random noise

    # Simulate some AD-like changes (very simplified)
    for i in range(num_ad):
        #  -  Increased "atrophy" (darker regions) in the center
        center_x, center_y, center_z = img_size[0] // 2, img_size[1] // 2, img_size[2] // 2
        radius = min(img_size) // 4
        for x in range(img_size[0]):
            for y in range(img_size[1]):
                for z in range(img_size[2]):
                    if (x - center_x) ** 2 + (y - center_y) ** 2 + (z - center_z) ** 2 <= radius**2:
                        images[i, x, y, z] *= 0.7  # Make it darker

        #  -  Increased "noise"
        images[i] += np.random.normal(0, 0.1, img_size)

    labels = np.array([1] * num_ad + [0] * num_control)  # 1 for AD, 0 for Control
    return images, labels


def save_as_nifti(images, labels, output_dir="synthetic_data"):
    """
    Saves the generated synthetic data as NIfTI files.

    Args:
        images (numpy.ndarray): Array of 3D images.
        labels (numpy.ndarray): Array of labels.
        output_dir (str): Directory to save the files.
    """

    import os
    os.makedirs(output_dir, exist_ok=True)

    for i in range(len(images)):
        img = nib.Nifti1Image(images[i], np.eye(4))  # Create NIfTI image
        label = labels[i]
        nib.save(img, os.path.join(output_dir, f"sample_{i}_label_{label}.nii.gz"))


if __name__ == '__main__':
    num_samples = 358  # As in the paper
    img_size = (64, 64, 64)  # Example size
    images, labels = generate_synthetic_mri_data(num_samples, img_size, ad_proportion=0.5)

    print(f"Generated {len(images)} synthetic samples.")
    print(f"Image shape: {images.shape[1:]}")
    print(f"Labels: {np.unique(labels)}")

    #  Example:  Save the data (optional)
    save_as_nifti(images, labels)
    print(f"Saved synthetic data to 'synthetic_data' directory")


    # --- Visualization Example (Optional - Requires matplotlib) ---
    # import matplotlib.pyplot as plt
    # example_image = images[0]
    # fig, axes = plt.subplots(4, 4, figsize=(8, 8))
    # for i in range(16):
    #     axes[i // 4, i % 4].imshow(example_image[:, :, i * (img_size[2] // 16)], cmap='gray')
    #     axes[i // 4, i % 4].axis('off')
    # plt.tight_layout()
    # plt.show()

Generated 358 synthetic samples.
Image shape: (64, 64, 64)
Labels: [0 1]
Saved synthetic data to 'synthetic_data' directory
time: 48.6 s (started: 2025-05-04 19:09:46 +00:00)


In [23]:
# -*- coding: utf-8 -*-
"""
Code to report the learning rate, batch size, hidden size, and dropout
used for the Alzheimer's Disease classification model.
"""

import numpy as np
import nibabel as nib  # For handling NIfTI image files
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    balanced_accuracy_score,
    roc_auc_score,
    accuracy_score,  # Accuracy
    average_precision_score,  # APS
    recall_score,  # Recall
    precision_score,  # Precision
)
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from scipy.ndimage import zoom  # For resizing images

# ----------------------------------------------------------------------
# 1. Data Loading and Preprocessing (Using GENERATED DATA)
# ----------------------------------------------------------------------

def load_synthetic_data(num_samples=358, img_size=(64, 64, 64), ad_proportion=0.5):
    """
    Generates synthetic MRI data.
    """
    num_ad = int(num_samples * ad_proportion)
    num_control = num_samples - num_ad
    images = np.random.rand(num_samples, img_size[0], img_size[1], img_size[2])
    for i in range(num_ad):
        center_x, center_y, center_z = img_size[0] // 2, img_size[1] // 2, img_size[2] // 2
        radius = min(img_size) // 4
        for x in range(img_size[0]):
            for y in range(img_size[1]):
                for z in range(img_size[2]):
                    if (x - center_x) ** 2 + (y - center_y) ** 2 + (z - center_z) ** 2 <= radius**2:
                        images[i, x, y, z] *= 0.7
        images[i] += np.random.normal(0, 0.1, img_size)
    labels = np.array([1] * num_ad + [0] * num_control)
    return images, labels


# Generate data
images, labels = load_synthetic_data(num_samples=358, img_size=(64, 64, 64), ad_proportion=0.5)
images = np.expand_dims(images, axis=-1)  # Add channel dimension

# Normalize images (example)
images = (images - np.mean(images)) / np.std(images)

# Split data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


# ----------------------------------------------------------------------
# 2. Model Definition and Hyperparameters
# ----------------------------------------------------------------------

# Define the hyperparameters
LEARNING_RATE = 0.001
BATCH_SIZE = 32
HIDDEN_SIZE = 64
DROPOUT_RATE = 0.5

def create_cnn_model(input_shape, hidden_size, dropout_rate, l2_reg=0.01):
    """
    Creates a simple 3D CNN model with configurable hyperparameters.
    """
    model = models.Sequential()
    model.add(layers.Conv3D(8, (3, 3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=l2(l2_reg)))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(16, (3, 3, 3), activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hidden_size, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

input_shape = X_train.shape[1:]
cnn_model = create_cnn_model(input_shape, HIDDEN_SIZE, DROPOUT_RATE)

# Compile the model
cnn_model.compile(optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

cnn_model.summary()

# ----------------------------------------------------------------------
# 3. Model Training
# ----------------------------------------------------------------------

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = cnn_model.fit(X_train, y_train,
                      epochs=1,
                      batch_size=BATCH_SIZE,
                      validation_split=0.1,
                      callbacks=[early_stopping])


# ----------------------------------------------------------------------
# 4. Model Evaluation
# ----------------------------------------------------------------------

# Evaluate the model
y_pred_prob = cnn_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

balanced_acc = balanced_accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_prob)
acc = accuracy_score(y_test, y_pred)
aps = average_precision_score(y_test, y_pred_prob)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)


print("\n--- Hyperparameter Report ---")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Hidden Layer Size: {HIDDEN_SIZE}")
print(f"Dropout Rate: {DROPOUT_RATE}")
print("\n--- Performance Evaluation ---")
print(f"Balanced Accuracy: {balanced_acc}")
print(f"AUC: {auc}")
print(f"Accuracy: {acc}")
print(f"Average Precision Score: {aps}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")

print("\nCode execution complete.")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_44 (Conv3D)              │ (None, 62, 62, 62, 8)  │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_44 (MaxPooling3D) │ (None, 31, 31, 31, 8)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_45 (Conv3D)              │ (None, 29, 29, 29, 16) │         3,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_45 (MaxPooling3D) │ (None, 14, 14, 14, 16) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_46 (Conv3D)              │ (None, 12, 12, 12, 32) │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_46 (MaxPooling3D) │ (None, 6, 6, 6, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 6912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 64)             │       442,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 460,049 (1.75 MB)

 Trainable params: 460,049 (1.75 MB)

 Non-trainable params: 0 (0.00 B)

9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 354ms/step - accuracy: 0.4799 - loss: 2.6094 - val_accuracy: 0.9655 - val_loss: 1.7837
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step

--- Hyperparameter Report ---
Learning Rate: 0.001
Batch Size: 32
Hidden Layer Size: 64
Dropout Rate: 0.5

--- Performance Evaluation ---
Balanced Accuracy: 0.8421052631578947
AUC: 0.9899380804953559
Accuracy: 0.8333333333333334
Average Precision Score: 0.9925438596491228
Recall: 0.6842105263157895
Precision: 1.0

Code execution complete.
time: 28.2 s (started: 2025-05-04 19:32:58 +00:00)


In [25]:
def create_cnn_model_ablation(input_shape, hidden_size, dropout_rate, num_conv_layers=3):
        """
        Creates a 3D CNN model with a variable number of convolutional layers.
        """
        model = models.Sequential()
        model.add(layers.Input(shape=input_shape))

        # Convolutional layers (variable number)
        for i in range(num_conv_layers):
            model.add(layers.Conv3D(8 * (2**i), (3, 3, 3), activation='relu', kernel_regularizer=l2(0.01)))
            model.add(layers.MaxPooling3D((2, 2, 2)))

        model.add(layers.Flatten())
        model.add(layers.Dense(hidden_size, activation='relu', kernel_regularizer=l2(0.01)))
        model.add(layers.Dropout(dropout_rate))
        model.add(layers.Dense(1, activation='sigmoid'))
        return model

# Example Usage:
input_shape = X_train.shape[1:]
# Ablation: Try different numbers of convolutional layers
for num_conv in [1, 2, 3]:
    print(f"\n--- Training with {num_conv} Conv Layers ---")
    cnn_model_ablation = create_cnn_model_ablation(input_shape, HIDDEN_SIZE, DROPOUT_RATE, num_conv_layers=num_conv)
    cnn_model_ablation.compile(optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
                              loss='binary_crossentropy',
                              metrics=['accuracy'])
    cnn_model_ablation.fit(X_train, y_train, epochs=1, batch_size=BATCH_SIZE, validation_split=0.1,
                          callbacks=[early_stopping])
    # ... (Evaluate and report results) ...
    y_pred_prob = cnn_model_ablation.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)

    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_prob)
    acc = accuracy_score(y_test, y_pred)
    aps = average_precision_score(y_test, y_pred_prob)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)


    print("\n--- Hyperparameter Report ---")
    print(f"Learning Rate: {LEARNING_RATE}")
    print(f"Batch Size: {BATCH_SIZE}")
    print(f"Hidden Layer Size: {HIDDEN_SIZE}")
    print(f"Dropout Rate: {DROPOUT_RATE}")
    print("\n--- Performance Evaluation ---")
    print(f"Balanced Accuracy: {balanced_acc}")
    print(f"AUC: {auc}")
    print(f"Accuracy: {acc}")
    print(f"Average Precision Score: {aps}")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")


--- Training with 1 Conv Layers ---
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 249ms/step - accuracy: 0.4869 - loss: 36.7824 - val_accuracy: 0.5172 - val_loss: 2.6922
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step

--- Hyperparameter Report ---
Learning Rate: 0.001
Batch Size: 32
Hidden Layer Size: 64
Dropout Rate: 0.5

--- Performance Evaluation ---
Balanced Accuracy: 0.5
AUC: 1.0
Accuracy: 0.5277777777777778
Average Precision Score: 1.0
Recall: 1.0
Precision: 0.5277777777777778

--- Training with 2 Conv Layers ---
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 321ms/step - accuracy: 0.5996 - loss: 3.6975 - val_accuracy: 1.0000 - val_loss: 1.3889
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step

--- Hyperparameter Report ---
Learning Rate: 0.001
Batch Size: 32
Hidden Layer Size: 64
Dropout Rate: 0.5

--- Performance Evaluation ---
Balanced Accuracy: 1.0
AUC: 1.0
Accuracy: 1.0
Average Precision Score: 1.0
Recall: 1.0
Precision: 1.0

--- Training with 3 Conv Layers ---
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 329ms/step - accuracy: 0.4670 - loss: 2.6